# GETTIN
## Planejamento de Equipamentos Escolares
## Método: Capacitaded Facility Location
### Versão: 0.9
#### Fillipe O Feitosa <fillipefeitosa@ua.pt>

---


#### Math Programming


\begin{split}& \text{minimizar} \quad    & \sum_{j=1}^m f_j x_j +\sum_{i=1}^n \sum_{j=1}^m c_{ij} y_{ij} &     \\
& \text{sujeito a:} \quad & \sum_{j=1}^m y_{ij} =d_i  &  \mbox{ for }  i=1,\cdots,n \\
&    & \sum_{i=1}^n y_{ij} \leq M_j x_j & \mbox{ para } j=1,\cdots,m  \\
&    & y_{ij} \leq d_i x_j              & \mbox{ para } i=1,\cdots,n; j=1,\cdots,m \\
&    & y_{ij} \geq 0                    & \mbox{ para } i=1,\cdots,n; j=1,\cdots,m \\
&    & x_j \in \{ 0,1 \}                & \mbox{ para } j=1,\cdots,m\end{split}

#### List of Imports

In [1]:
# Import Libraries
from gurobi import *
import math
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import geopy.distance
from collections import defaultdict
# TEST Used for testing with a small sample
import random


#### Problem Data

In [2]:
# ------------------------- Problem data ------------------------- #

# Dict for SubSections With Demand
rawData = pd.read_excel('./subseccao_vagos/Pop_escolar_Vagos.xlsx')
listOfSubsections_2010 = {}
for e in range(len(rawData)-1):
    listOfSubsections_2010[int(rawData.SUBSECCAO[e])] = rawData.P_esc_1CEB_2010[e]

    
# TEST Used _sample for testing with a small sample to CREATE A SAMPLE OF POTENTIAL SCHOOL POINTS   
# listOfSubsections_2010_sample = {}

# for i in range(80):
#     sampleSchool, sampleDemand = random.choice(list(listOfSubsections_2010.items()))
#     # If statement: Do not repeat random schools
#     if sampleSchool not in listOfSubsections_2010_sample.keys():
#         listOfSubsections_2010_sample[sampleSchool] = sampleDemand
        

# Dict of Lists for Schools with Capacity And Cost by Size
tupleOfCapacity = (21, 80, 225)

# TEST used for testing with small sample
# tupleOfCapacity = (10, 20, 50)

# A list of all possible schools and its size, plus cost of Instalation
schoolsCapacityCost = defaultdict(list)

# # TEST Using _SAMPLE to get a sample of Points to create schools
for bgri in list(listOfSubsections_2010.keys()):
    for capacity in tupleOfCapacity:
        #  Assuming Default Values
        if(capacity==21):
            cost = 700000
            costPerStudent = 5275.22
        elif(capacity==80):
            cost = 1200000
            costPerStudent = 2750.15
        elif(capacity==225):
            cost = 1800000
            costPerStudent = 1662
        # cost = -17.831*capacity + 4305.1
        tempList = (capacity, cost, costPerStudent)
        schoolsCapacityCost[bgri].append(tempList)


# Retrieve Coordinates and Vinculate to Identifier (BGRI)
vagos = gpd.read_file('./data_gettin/vagos.geojson');
centroids = vagos.centroid

iteratorHandler = centroids.size
centroidVector = []
for centroid in centroids:
    obj = [centroid.xy[0][0], centroid.xy[1][0]]
    centroidVector.append(obj)
    
coordinates_X = {}
coordinates_Y = {}
for e in range (len(vagos)):
    coordinates_X[int(vagos.BGRI11[e])] = centroids[e].xy[0][0]
    coordinates_Y[int(vagos.BGRI11[e])] = centroids[e].xy[1][0]
    len
coordinates_X[list(listOfSubsections_2010.keys())[0]]

-8.69434885302111

In [3]:
# if 1180400403 not in listOfSubsections_2010_sample.keys():
#     print("nao está")
# sum(listOfSubsections_2010.values())

# schoolsCapacityCost
schoolsCapacityCost[1180100101][0][0]

21

#### Modeling and creation of decision Variables

In [4]:
   
# -------- Distance Matrix Logic ------- #
transportationCosts = {}

for bgri_1 in list(listOfSubsections_2010.keys()):
    coords_1 = (coordinates_X[bgri_1], coordinates_Y[bgri_1])
    for bgri_2 in list(listOfSubsections_2010.keys()):
        coords_2 = (coordinates_X[bgri_2], coordinates_Y[bgri_2])
        # 2 * 0.36 cents per KM * 180 days * 10 years
        cost = 2*geopy.distance.geodesic(coords_1, coords_2).km*0.36*180*10
        transportationCosts[bgri_1, bgri_2] = cost
        

transportationCosts


{(1180100101, 1180100101): 0.0,
 (1180100101, 1180100102): 267.4046356857272,
 (1180100101, 1180100103): 375.9191387594541,
 (1180100101, 1180100104): 941.6484959791135,
 (1180100101, 1180100105): 617.3815915063698,
 (1180100101, 1180100106): 1287.4533642389817,
 (1180100101, 1180100107): 1602.2318309286452,
 (1180100101, 1180100108): 1530.6027320147712,
 (1180100101, 1180100109): 2108.6077530782877,
 (1180100101, 1180100110): 1663.414276135425,
 (1180100101, 1180100111): 1863.1045716399135,
 (1180100101, 1180100112): 1991.4828945696431,
 (1180100101, 1180100113): 1888.240689609458,
 (1180100101, 1180100114): 2131.431781441174,
 (1180100101, 1180100115): 2843.506571286539,
 (1180100101, 1180100116): 2664.324676497611,
 (1180100101, 1180100117): 939.3089526776463,
 (1180100101, 1180100118): 1638.5352561423756,
 (1180100101, 1180100119): 1345.5232638987434,
 (1180100101, 1180100120): 2028.9958517325965,
 (1180100101, 1180100121): 1361.5031643440893,
 (1180100101, 1180100122): 1722.443509

In [5]:
(transportationCosts[(1180800409,1180400203)])/(2*0.36*180*10)

5.576519696031139

In [6]:
# for school in list(schoolsCapacityCost.keys()):
#     for schoolSize in range(len(schoolsCapacityCost[school])):
#         print(schoolsCapacityCost[school][schoolSize][0])
#         print(schoolSize)

In [7]:

# -------- Decision Variables ---------- #

# numSchools = len(escola)
# numSubSections = len(subSecao)

# Creting Guroby Model
m = Model()

# Decision Variables
x = {}
y = {}

# creating binary variable for every school
for j in list(schoolsCapacityCost.keys()):
    for schoolSize in tupleOfCapacity:
        x[(j,schoolSize)] = m.addVar(vtype=GRB.BINARY, name="(%d,%d)" % (j, schoolSize))

# creating continuous variable for subsections to check suply fractions
for subsection in list(listOfSubsections_2010.keys()):
    for school in list(schoolsCapacityCost.keys()):
        for capacity in schoolsCapacityCost[school]:
        # y of Subsection Suply
            y[(subsection,school,capacity[0])] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Fração da Sub[%d], escola[%d][%d]" % (subsection,school, capacity[0]))

m.update()

Academic license - for non-commercial use only


In [8]:
# schoolsCapacityCost[1180100101][1]
# listOfSubsections_2010[1180100101]
# for school in list(schoolsCapacityCost.keys()):
#     for schoolSize in range(len(schoolsCapacityCost[school])):
#         sizeOfSchool = schoolsCapacityCost[school][schoolSize][0]
#         print(x[school, sizeOfSchool])
# for capacity in schoolsCapacityCost[1180800306]:
#     print(capacity[0])

# for capacity in tupleOfCapacity:
#     print(capacity)
    
# for school in schoolsCapacityCost:
#     resultCache = [item for item in x if item[0] == school]
#     print(resultCache)
#     for e in resultCache:
#         print(str(e))

# for school in schoolsCapacityCost.keys():
#     print(x[(school, k)] for k in tupleOfCapacity)
#     m.addConstr(quicksum)

# for element in x:
#     print(element)

# len(x[(1180700102, 120)])
# m.getVars()
# for school in schoolsCapacityCost:
#     resultCache = [item for item in x if item[0] == school]
# #     m.addConstr( <= 1, "SameSiteRestriction(%s)"%(resultCache))
#     print(x[resultCache[1]])


# for i in range(len(resultCache)):
#     print(x[resultCache[i]])

# resultado = x[resultCache[0]]+x[resultCache[1]] + x[resultCache[2]]
# print(resultado)

## Adding Constraints

In [9]:
# Constraint for Every Student on School
# for i in range(numSubSections):
#     m.addConstr(quicksum(y[(i,j)] for j in range(numSchools)) == 1)

# Constraint to repect demands on Every Subsection - OK
for i in list(listOfSubsections_2010.keys()):
    m.addConstr(quicksum(y[(i,j,k)] for j in list(schoolsCapacityCost.keys()) for k in tupleOfCapacity) == listOfSubsections_2010[i])
    
# Constraint to restrain school capacity - OK
for school in list(schoolsCapacityCost.keys()):
    for schoolSize in range(len(schoolsCapacityCost[school])):
        numericalSizeOfSchool = schoolsCapacityCost[school][schoolSize][0]
        m.addConstr(quicksum(y[(i,school, numericalSizeOfSchool)] for i in list(listOfSubsections_2010.keys())) <= numericalSizeOfSchool * x[school, numericalSizeOfSchool], "SchoolK(%s,%s)"%(school, numericalSizeOfSchool))
    
# Add restriction to fraction Size - OK
for (i,j,k) in y:
    m.addConstr(y[i,j,k] <= listOfSubsections_2010[i]*x[j, k], "FractionConst(%s,%s,%s)"%(i,j,k))

# Restriction to exclude multiple schools being constructed on the same site
for school in schoolsCapacityCost:
    resultCache = [item for item in x if item[0] == school]
    # HardCoded The Sum of Results to That Group of Schools Range(3) [0,1,2]
    cacheQuickSum = x[resultCache[0]]+x[resultCache[1]]+x[resultCache[2]]
    m.addConstr( cacheQuickSum <= 1, "SameSiteRestriction(%s)"%(cacheQuickSum))

In [10]:
schoolsCapacityCost

defaultdict(list,
            {1180100101: [(21, 700000, 5275.22),
              (80, 1200000, 2750.15),
              (225, 1800000, 1662)],
             1180100102: [(21, 700000, 5275.22),
              (80, 1200000, 2750.15),
              (225, 1800000, 1662)],
             1180100103: [(21, 700000, 5275.22),
              (80, 1200000, 2750.15),
              (225, 1800000, 1662)],
             1180100104: [(21, 700000, 5275.22),
              (80, 1200000, 2750.15),
              (225, 1800000, 1662)],
             1180100105: [(21, 700000, 5275.22),
              (80, 1200000, 2750.15),
              (225, 1800000, 1662)],
             1180100106: [(21, 700000, 5275.22),
              (80, 1200000, 2750.15),
              (225, 1800000, 1662)],
             1180100107: [(21, 700000, 5275.22),
              (80, 1200000, 2750.15),
              (225, 1800000, 1662)],
             1180100108: [(21, 700000, 5275.22),
              (80, 1200000, 2750.15),
              (225, 1800000

#### Objetive


In [11]:
# Setting objective

m.setObjective(
    quicksum(schoolsCapacityCost[school][schoolSize][1]*x[school, schoolsCapacityCost[school][schoolSize][0]] for school in list(schoolsCapacityCost.keys()) for schoolSize in range(len(schoolsCapacityCost[school]))) + 
    quicksum(transportationCosts[i,j]*y[i,j,k] for i in list(listOfSubsections_2010.keys()) for j in list(schoolsCapacityCost.keys()) for k in tupleOfCapacity), GRB.MINIMIZE )
# IF we are going to add some operation costs defined for the number of students, should be here
# (schoolsCapacityCost[j][schoolSize][2]+transportationCosts[i,j])

In [12]:
m.optimize()

Optimize a model with 1224322 rows, 1223046 columns and 4668246 nonzeros
Variable types: 1221132 continuous, 1914 integer (1914 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+02]
  Objective range  [4e+01, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+01]
Found heuristic solution: objective 5.163452e+08
Presolve removed 220225 rows and 220110 columns (presolve time = 6s) ...
Presolve removed 220225 rows and 220110 columns (presolve time = 10s) ...
Presolve removed 220225 rows and 220110 columns (presolve time = 15s) ...
Presolve removed 220225 rows and 220110 columns (presolve time = 24s) ...
Presolve removed 220225 rows and 220110 columns (presolve time = 28s) ...
Presolve removed 220225 rows and 220110 columns (presolve time = 31s) ...
Presolve removed 220225 rows and 220110 columns
Presolve time: 30.81s
Presolved: 1004097 rows, 1002936 columns, 4007916 nonzeros
Variable types: 1001022 continuous, 1914 integer (1914 binary)

Deterministic concu

In [13]:
m.getConstrs()

[<gurobi.Constr R0>,
 <gurobi.Constr R1>,
 <gurobi.Constr R2>,
 <gurobi.Constr R3>,
 <gurobi.Constr R4>,
 <gurobi.Constr R5>,
 <gurobi.Constr R6>,
 <gurobi.Constr R7>,
 <gurobi.Constr R8>,
 <gurobi.Constr R9>,
 <gurobi.Constr R10>,
 <gurobi.Constr R11>,
 <gurobi.Constr R12>,
 <gurobi.Constr R13>,
 <gurobi.Constr R14>,
 <gurobi.Constr R15>,
 <gurobi.Constr R16>,
 <gurobi.Constr R17>,
 <gurobi.Constr R18>,
 <gurobi.Constr R19>,
 <gurobi.Constr R20>,
 <gurobi.Constr R21>,
 <gurobi.Constr R22>,
 <gurobi.Constr R23>,
 <gurobi.Constr R24>,
 <gurobi.Constr R25>,
 <gurobi.Constr R26>,
 <gurobi.Constr R27>,
 <gurobi.Constr R28>,
 <gurobi.Constr R29>,
 <gurobi.Constr R30>,
 <gurobi.Constr R31>,
 <gurobi.Constr R32>,
 <gurobi.Constr R33>,
 <gurobi.Constr R34>,
 <gurobi.Constr R35>,
 <gurobi.Constr R36>,
 <gurobi.Constr R37>,
 <gurobi.Constr R38>,
 <gurobi.Constr R39>,
 <gurobi.Constr R40>,
 <gurobi.Constr R41>,
 <gurobi.Constr R42>,
 <gurobi.Constr R43>,
 <gurobi.Constr R44>,
 <gurobi.Constr R45>

In [14]:
print('Obj: %g' % m.objVal)

Obj: 5.16345e+08


In [15]:
resultsBgriCapacity = []
resultsSubFractionToSchool = []
for v in m.getVars():
    if(v.x != 0):
        print('%s   %g' % (v.varName, v.x))
        if(v.varName.startswith('(')):
            resultsBgriCapacity.append(v.varName)        
        else:
            resultsSubFractionToSchool.append(v.varName)

(1180100101,80)   1
(1180100102,80)   1
(1180100103,80)   1
(1180100104,225)   1
(1180100105,80)   1
(1180100106,80)   1
(1180100107,21)   1
(1180100108,21)   1
(1180100110,80)   1
(1180100111,80)   1
(1180100112,21)   1
(1180100114,21)   1
(1180100116,225)   1
(1180100117,21)   1
(1180100118,21)   1
(1180100119,80)   1
(1180100120,80)   1
(1180100121,225)   1
(1180100124,80)   1
(1180100125,80)   1
(1180100126,80)   1
(1180100127,225)   1
(1180100129,80)   1
(1180100132,225)   1
(1180100202,225)   1
(1180100204,80)   1
(1180100205,80)   1
(1180100209,80)   1
(1180100210,80)   1
(1180100211,225)   1
(1180100215,21)   1
(1180100216,80)   1
(1180100217,225)   1
(1180100220,225)   1
(1180100222,225)   1
(1180100223,21)   1
(1180100225,80)   1
(1180100226,21)   1
(1180100227,21)   1
(1180100228,80)   1
(1180100229,225)   1
(1180100230,225)   1
(1180100231,21)   1
(1180100232,80)   1
(1180100233,225)   1
(1180200101,21)   1
(1180200102,21)   1
(1180200103,21)   1
(1180200106,225)   1
(11802

Fração da Sub[1180100120], escola[1181100124][21]   2.71582
Fração da Sub[1180100121], escola[1180500105][80]   6.20759
Fração da Sub[1180100122], escola[1180800506][80]   0.775949
Fração da Sub[1180100123], escola[1180800608][21]   0.193987
Fração da Sub[1180100124], escola[1180900116][21]   0.387975
Fração da Sub[1180100125], escola[1180900203][225]   2.90981
Fração da Sub[1180100126], escola[1180100126][80]   0.969936
Fração da Sub[1180100127], escola[1180600119][21]   1.35791
Fração da Sub[1180100128], escola[1180100114][21]   1.74589
Fração da Sub[1180100129], escola[1180900109][225]   0.969936
Fração da Sub[1180100201], escola[1180600205][225]   1.16392
Fração da Sub[1180100202], escola[1180700307][80]   0.193987
Fração da Sub[1180100203], escola[1181000102][80]   0.193987
Fração da Sub[1180100204], escola[1180700104][21]   3.87975
Fração da Sub[1180100205], escola[1180700415][80]   2.90981
Fração da Sub[1180100207], escola[1180400804][80]   0.581962
Fração da Sub[1180100209], es

Fração da Sub[1180400301], escola[1180600206][225]   4.84968
Fração da Sub[1180400302], escola[1180800604][80]   5.04367
Fração da Sub[1180400303], escola[1180100231][80]   4.88498e-15
Fração da Sub[1180400303], escola[1181000215][21]   3.68576
Fração da Sub[1180400304], escola[1180200114][80]   2.32785
Fração da Sub[1180400305], escola[1180200233][80]   8.92341
Fração da Sub[1180400306], escola[1180800205][225]   0.581962
Fração da Sub[1180400308], escola[1180100117][21]   0.387975
Fração da Sub[1180400310], escola[1181000115][80]   0.581962
Fração da Sub[1180400401], escola[1180800108][80]   3.29778
Fração da Sub[1180400402], escola[1180900104][225]   3.29778
Fração da Sub[1180400403], escola[1180200115][225]   0.581962
Fração da Sub[1180400404], escola[1180700313][80]   1.93987
Fração da Sub[1180400501], escola[1180300105][225]   1.93987
Fração da Sub[1180400503], escola[1181000103][80]   7.77156e-16
Fração da Sub[1180400503], escola[1181000105][21]   0.969936
Fração da Sub[11804005

Fração da Sub[1180600218], escola[1180400811][80]   2.13386
Fração da Sub[1180600219], escola[1181000303][21]   2.32785
Fração da Sub[1180600220], escola[1181000211][21]   0.969936
Fração da Sub[1180600221], escola[1180600212][21]   1.93987
Fração da Sub[1180600222], escola[1180300105][225]   0.775949
Fração da Sub[1180600223], escola[1181100133][225]   2.71582
Fração da Sub[1180600224], escola[1180600214][225]   2.13386
Fração da Sub[1180600225], escola[1180600121][80]   1.74589
Fração da Sub[1180600226], escola[1180600203][21]   4.46171
Fração da Sub[1180600227], escola[1180700301][80]   0.193987
Fração da Sub[1180700101], escola[1180600101][225]   1.93987
Fração da Sub[1180700102], escola[1180900116][21]   6.01361
Fração da Sub[1180700103], escola[1180800108][80]   7.95348
Fração da Sub[1180700104], escola[1181000120][80]   1.74589
Fração da Sub[1180700105], escola[1180500113][80]   1.35791
Fração da Sub[1180700106], escola[1180200116][225]   1.16392
Fração da Sub[1180700107], escol

Fração da Sub[1180800606], escola[1180800508][225]   2.52183
Fração da Sub[1180800607], escola[1181100103][225]   7.5655
Fração da Sub[1180800608], escola[1180600119][21]   7.17753
Fração da Sub[1180800609], escola[1180100114][21]   2.13386
Fração da Sub[1180800611], escola[1180500106][80]   0.193987
Fração da Sub[1180800612], escola[1180800308][21]   3.49177
Fração da Sub[1180800614], escola[1180400802][80]   0.775949
Fração da Sub[1180800615], escola[1180800414][21]   2.32785
Fração da Sub[1180800616], escola[1180100231][21]   2.71582
Fração da Sub[1180900101], escola[1181000307][21]   2.52183
Fração da Sub[1180900102], escola[1180700311][225]   4.07373
Fração da Sub[1180900103], escola[1180800408][225]   4.65569
Fração da Sub[1180900104], escola[1181000106][80]   2.71582
Fração da Sub[1180900105], escola[1181000116][21]   4.44089e-16
Fração da Sub[1180900105], escola[1181000212][21]   3.29778
Fração da Sub[1180900106], escola[1180800411][225]   1.16392
Fração da Sub[1180900107], esc

In [16]:
len(resultsBgriCapacity)

419

In [17]:
print(resultsBgriCapacity)

['(1180100101,80)', '(1180100102,80)', '(1180100103,80)', '(1180100104,225)', '(1180100105,80)', '(1180100106,80)', '(1180100107,21)', '(1180100108,21)', '(1180100110,80)', '(1180100111,80)', '(1180100112,21)', '(1180100114,21)', '(1180100116,225)', '(1180100117,21)', '(1180100118,21)', '(1180100119,80)', '(1180100120,80)', '(1180100121,225)', '(1180100124,80)', '(1180100125,80)', '(1180100126,80)', '(1180100127,225)', '(1180100129,80)', '(1180100132,225)', '(1180100202,225)', '(1180100204,80)', '(1180100205,80)', '(1180100209,80)', '(1180100210,80)', '(1180100211,225)', '(1180100215,21)', '(1180100216,80)', '(1180100217,225)', '(1180100220,225)', '(1180100222,225)', '(1180100223,21)', '(1180100225,80)', '(1180100226,21)', '(1180100227,21)', '(1180100228,80)', '(1180100229,225)', '(1180100230,225)', '(1180100231,21)', '(1180100232,80)', '(1180100233,225)', '(1180200101,21)', '(1180200102,21)', '(1180200103,21)', '(1180200106,225)', '(1180200110,21)', '(1180200111,80)', '(1180200114,80)

In [18]:
listOfSubsections_2010

{1180100101: 0.387974544656572,
 1180100102: 0.96993636164143,
 1180100103: 1.55189817862629,
 1180100104: 0.96993636164143,
 1180100105: 0.387974544656572,
 1180100106: 3.10379635725258,
 1180100107: 0.775949089313145,
 1180100108: 1.357910906298,
 1180100109: 0.775949089313145,
 1180100110: 0.0,
 1180100111: 2.52183454026772,
 1180100112: 0.775949089313145,
 1180100113: 0.0,
 1180100114: 1.93987272328286,
 1180100115: 1.16392363396972,
 1180100116: 0.387974544656572,
 1180100117: 1.93987272328286,
 1180100118: 0.387974544656572,
 1180100119: 1.74588545095457,
 1180100120: 2.71582181259601,
 1180100121: 6.20759271450516,
 1180100122: 0.775949089313145,
 1180100123: 0.193987272328286,
 1180100124: 0.387974544656572,
 1180100125: 2.90980908492429,
 1180100126: 0.96993636164143,
 1180100127: 1.357910906298,
 1180100128: 1.74588545095457,
 1180100129: 0.96993636164143,
 1180100131: 0.0,
 1180100132: 0.0,
 1180100201: 1.16392363396972,
 1180100202: 0.193987272328286,
 1180100203: 0.1939872

In [19]:
# Routine to calculate Distances
results = pd.ExcelFile('data_gettin/ResultadosParaArtigo.xlsx')
resultsCapacited = pd.read_excel(results, 'Capacitated')
resultsKmedian = pd.read_excel(results, 'KMedian')
int(resultsCapacited.iloc[0].SubSection)

capacitadedDistances = {}
for result in resultsCapacited.iterrows():
    capacitadedDistances[int(result[1].SubSection), int(result[1].School)] = transportationCosts[int(result[1].SubSection), int(result[1].School)]


kMedianDistances = {}
for result in resultsKmedian.iterrows():
    if(int(result[1].SubSection in listOfSubsections_2010.keys())):
        kMedianDistances[int(result[1].SubSection), int(result[1].School)] = transportationCosts[int(result[1].SubSection), int(result[1].School)]


In [20]:
listOfSubsections_2010[1180400310]

0.581961816984858

In [21]:
transportationCosts[1180400105, 1180200209]

KeyError: (1180400105, 1180200209)

In [ ]:
m

In [ ]:
m = None

In [ ]:
disposeDefaultEnv()

# 